In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5728,2020-11-22T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1597,241,1838,35324,...,921,1258,14781,1186,32768.0,20361.0,53129,882336,606566.0,NaN
5729,2020-11-22T17:00:00,ITA,9,Toscana,43.769231,11.255889,1785,298,2083,50641,...,-416,1929,39765,2216,73871.0,20834.0,94705,1457791,912549.0,NaN
5730,2020-11-22T17:00:00,ITA,10,Umbria,43.106758,12.388247,368,75,443,11129,...,149,400,9864,333,6133.0,15636.0,21769,390378,215754.0,NaN
5731,2020-11-22T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,144,14,158,1761,...,-82,99,3818,285,5364.0,658.0,6022,54841,31620.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5728,2020-11-22T17:00:00,19,Sicilia,1597,241,1838,35324,37162,921,1258,14781,1186,32768.0,20361.0,53129,882336,606566.0
5729,2020-11-22T17:00:00,9,Toscana,1785,298,2083,50641,52724,-416,1929,39765,2216,73871.0,20834.0,94705,1457791,912549.0
5730,2020-11-22T17:00:00,10,Umbria,368,75,443,11129,11572,149,400,9864,333,6133.0,15636.0,21769,390378,215754.0
5731,2020-11-22T17:00:00,2,Valle d'Aosta,144,14,158,1761,1919,-82,99,3818,285,5364.0,658.0,6022,54841,31620.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-22', '2020-11-21')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-22T17:00:00,1,Piemonte,5132,398,5530,71482,77012,920,2641,66998,5565,64386.0,85189.0,149575,1418692,898135.0
1,2020-11-22T17:00:00,2,Valle d'Aosta,144,14,158,1761,1919,-82,99,3818,285,5364.0,658.0,6022,54841,31620.0
2,2020-11-22T17:00:00,3,Lombardia,8391,949,9340,155066,164406,1721,5094,186680,20524,276322.0,95288.0,371610,3798851,2248813.0
3,2020-11-22T17:00:00,5,Veneto,2211,281,2492,69964,72456,1709,2956,47005,3221,30237.0,92445.0,122682,2672240,1027321.0
4,2020-11-22T17:00:00,6,Friuli Venezia Giulia,545,56,601,12450,13051,294,675,10961,646,20886.0,3772.0,24658,664296,264942.0
5,2020-11-22T17:00:00,7,Liguria,1253,122,1375,14503,15878,5,611,30008,2241,33196.0,14931.0,48127,564766,279225.0
6,2020-11-22T17:00:00,8,Emilia-Romagna,2666,249,2915,64359,67274,2194,2665,33105,5350,66834.0,38895.0,105729,1990596,1027297.0
7,2020-11-22T17:00:00,9,Toscana,1785,298,2083,50641,52724,-416,1929,39765,2216,73871.0,20834.0,94705,1457791,912549.0
8,2020-11-22T17:00:00,10,Umbria,368,75,443,11129,11572,149,400,9864,333,6133.0,15636.0,21769,390378,215754.0
9,2020-11-22T17:00:00,11,Marche,542,88,630,14737,15367,245,529,10047,1180,20395.0,6199.0,26594,391100,227685.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-21T17:00:00,1,Piemonte,5150,390,5540,70552,76092,-453,2896,65346,5496,63328.0,83606.0,146934,1405087,890725.0
1,2020-11-21T17:00:00,2,Valle d'Aosta,149,11,160,1841,2001,-79,91,3641,281,5268.0,655.0,5923,53960,31251.0
2,2020-11-21T17:00:00,3,Lombardia,8314,936,9250,153435,162685,3044,8853,183472,20359,272904.0,93612.0,366516,3769051,2236696.0
3,2020-11-21T17:00:00,5,Veneto,2200,275,2475,68272,70747,2060,3567,45789,3190,29844.0,89882.0,119726,2658845,1022442.0
4,2020-11-21T17:00:00,6,Friuli Venezia Giulia,509,55,564,12193,12757,582,1043,10593,633,20317.0,3666.0,23983,658272,263440.0
5,2020-11-21T17:00:00,7,Liguria,1241,119,1360,14513,15873,-81,761,29416,2227,32708.0,14808.0,47516,560338,277654.0
6,2020-11-21T17:00:00,8,Emilia-Romagna,2594,241,2835,62245,65080,2146,2723,32672,5312,65480.0,37584.0,103064,1977482,1019829.0
7,2020-11-21T17:00:00,9,Toscana,1798,296,2094,51046,53140,-729,1892,37471,2165,72642.0,20134.0,92776,1440310,904959.0
8,2020-11-21T17:00:00,10,Umbria,370,75,445,10978,11423,55,408,9623,323,5995.0,15374.0,21369,385737,213635.0
9,2020-11-21T17:00:00,11,Marche,544,86,630,14492,15122,145,452,9775,1168,20054.0,6011.0,26065,387818,225892.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-22T17:00:00,1,Piemonte,5132,398,5530,71482,77012,920,2641,66998,5565,64386.0,85189.0,149575,1418692,898135.0
1,2020-11-22T17:00:00,2,Valle d'Aosta,144,14,158,1761,1919,-82,99,3818,285,5364.0,658.0,6022,54841,31620.0
2,2020-11-22T17:00:00,3,Lombardia,8391,949,9340,155066,164406,1721,5094,186680,20524,276322.0,95288.0,371610,3798851,2248813.0
3,2020-11-22T17:00:00,4,Trentino-Alto Adige,873,79,952,13214,14166,-211,694,20119,1047,28514.0,6818.0,35332,646944,272866.0
4,2020-11-22T17:00:00,5,Veneto,2211,281,2492,69964,72456,1709,2956,47005,3221,30237.0,92445.0,122682,2672240,1027321.0
5,2020-11-22T17:00:00,6,Friuli Venezia Giulia,545,56,601,12450,13051,294,675,10961,646,20886.0,3772.0,24658,664296,264942.0
6,2020-11-22T17:00:00,7,Liguria,1253,122,1375,14503,15878,5,611,30008,2241,33196.0,14931.0,48127,564766,279225.0
7,2020-11-22T17:00:00,8,Emilia-Romagna,2666,249,2915,64359,67274,2194,2665,33105,5350,66834.0,38895.0,105729,1990596,1027297.0
8,2020-11-22T17:00:00,9,Toscana,1785,298,2083,50641,52724,-416,1929,39765,2216,73871.0,20834.0,94705,1457791,912549.0
9,2020-11-22T17:00:00,10,Umbria,368,75,443,11129,11572,149,400,9864,333,6133.0,15636.0,21769,390378,215754.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-21T17:00:00,1,Piemonte,5150,390,5540,70552,76092,-453,2896,65346,5496,63328.0,83606.0,146934,1405087,890725.0
1,1,2020-11-21T17:00:00,2,Valle d'Aosta,149,11,160,1841,2001,-79,91,3641,281,5268.0,655.0,5923,53960,31251.0
2,2,2020-11-21T17:00:00,3,Lombardia,8314,936,9250,153435,162685,3044,8853,183472,20359,272904.0,93612.0,366516,3769051,2236696.0
3,19,2020-11-21T17:00:00,4,Trentino-Alto Adige,863,79,942,13435,14377,-138,760,19222,1039,27917.0,6721.0,34638,640520,271022.0
4,3,2020-11-21T17:00:00,5,Veneto,2200,275,2475,68272,70747,2060,3567,45789,3190,29844.0,89882.0,119726,2658845,1022442.0
5,4,2020-11-21T17:00:00,6,Friuli Venezia Giulia,509,55,564,12193,12757,582,1043,10593,633,20317.0,3666.0,23983,658272,263440.0
6,5,2020-11-21T17:00:00,7,Liguria,1241,119,1360,14513,15873,-81,761,29416,2227,32708.0,14808.0,47516,560338,277654.0
7,6,2020-11-21T17:00:00,8,Emilia-Romagna,2594,241,2835,62245,65080,2146,2723,32672,5312,65480.0,37584.0,103064,1977482,1019829.0
8,7,2020-11-21T17:00:00,9,Toscana,1798,296,2094,51046,53140,-729,1892,37471,2165,72642.0,20134.0,92776,1440310,904959.0
9,8,2020-11-21T17:00:00,10,Umbria,370,75,445,10978,11423,55,408,9623,323,5995.0,15374.0,21369,385737,213635.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-22T17:00:00,1,Piemonte,5132,398,5530,71482,77012,920,2641,66998,5565,64386.0,85189.0,149575,1418692,898135.0,4341375
1,2020-11-22T17:00:00,2,Valle d'Aosta,144,14,158,1761,1919,-82,99,3818,285,5364.0,658.0,6022,54841,31620.0,125501
2,2020-11-22T17:00:00,3,Lombardia,8391,949,9340,155066,164406,1721,5094,186680,20524,276322.0,95288.0,371610,3798851,2248813.0,10103969
3,2020-11-22T17:00:00,4,Trentino-Alto Adige,873,79,952,13214,14166,-211,694,20119,1047,28514.0,6818.0,35332,646944,272866.0,1074819
4,2020-11-22T17:00:00,5,Veneto,2211,281,2492,69964,72456,1709,2956,47005,3221,30237.0,92445.0,122682,2672240,1027321.0,4907704
5,2020-11-22T17:00:00,6,Friuli Venezia Giulia,545,56,601,12450,13051,294,675,10961,646,20886.0,3772.0,24658,664296,264942.0,1211357
6,2020-11-22T17:00:00,7,Liguria,1253,122,1375,14503,15878,5,611,30008,2241,33196.0,14931.0,48127,564766,279225.0,1543127
7,2020-11-22T17:00:00,8,Emilia-Romagna,2666,249,2915,64359,67274,2194,2665,33105,5350,66834.0,38895.0,105729,1990596,1027297.0,4467118
8,2020-11-22T17:00:00,9,Toscana,1785,298,2083,50641,52724,-416,1929,39765,2216,73871.0,20834.0,94705,1457791,912549.0,3722729
9,2020-11-22T17:00:00,10,Umbria,368,75,443,11129,11572,149,400,9864,333,6133.0,15636.0,21769,390378,215754.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-22T17:00:00,1,Piemonte,5132,398,5530,71482,77012,920,2641,66998,5565,64386.0,85189.0,149575,1418692,898135.0,4341375,575
1,2020-11-22T17:00:00,2,Valle d'Aosta,144,14,158,1761,1919,-82,99,3818,285,5364.0,658.0,6022,54841,31620.0,125501,20
2,2020-11-22T17:00:00,3,Lombardia,8391,949,9340,155066,164406,1721,5094,186680,20524,276322.0,95288.0,371610,3798851,2248813.0,10103969,1036
3,2020-11-22T17:00:00,4,Trentino-Alto Adige,873,79,952,13214,14166,-211,694,20119,1047,28514.0,6818.0,35332,646944,272866.0,1074819,106
4,2020-11-22T17:00:00,5,Veneto,2211,281,2492,69964,72456,1709,2956,47005,3221,30237.0,92445.0,122682,2672240,1027321.0,4907704,1016
5,2020-11-22T17:00:00,6,Friuli Venezia Giulia,545,56,601,12450,13051,294,675,10961,646,20886.0,3772.0,24658,664296,264942.0,1211357,180
6,2020-11-22T17:00:00,7,Liguria,1253,122,1375,14503,15878,5,611,30008,2241,33196.0,14931.0,48127,564766,279225.0,1543127,209
7,2020-11-22T17:00:00,8,Emilia-Romagna,2666,249,2915,64359,67274,2194,2665,33105,5350,66834.0,38895.0,105729,1990596,1027297.0,4467118,634
8,2020-11-22T17:00:00,9,Toscana,1785,298,2083,50641,52724,-416,1929,39765,2216,73871.0,20834.0,94705,1457791,912549.0,3722729,523
9,2020-11-22T17:00:00,10,Umbria,368,75,443,11129,11572,149,400,9864,333,6133.0,15636.0,21769,390378,215754.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-22T17:00:00,3,Lombardia,8391,949,9340,155066,164406,1721,5094,...,91.60,1631,-3759,29800,12117.0,165,3208,17.09,0.05042,3.68
1,2020-11-22T17:00:00,15,Campania,2218,201,2419,101062,103481,2346,3217,...,39.80,2385,-337,24332,17694.0,21,850,13.22,0.05560,2.36
2,2020-11-22T17:00:00,5,Veneto,2211,281,2492,69964,72456,1709,2956,...,27.66,1692,-611,13395,4879.0,31,1216,22.07,0.06023,2.50
3,2020-11-22T17:00:00,8,Emilia-Romagna,2666,249,2915,64359,67274,2194,2665,...,39.27,2114,-58,13114,7468.0,38,433,20.32,0.05966,2.37
4,2020-11-22T17:00:00,1,Piemonte,5132,398,5530,71482,77012,920,2641,...,69.22,930,-255,13605,7410.0,69,1652,19.41,0.06083,3.45
5,2020-11-22T17:00:00,12,Lazio,3298,339,3637,78416,82053,2177,2533,...,40.02,2134,-125,24643,19410.0,20,336,10.28,0.04318,1.75
6,2020-11-22T17:00:00,9,Toscana,1785,298,2083,50641,52724,-416,1929,...,56.98,-405,37,17481,7590.0,51,2294,11.03,0.05182,2.54
7,2020-11-22T17:00:00,16,Puglia,1449,188,1637,29715,31352,932,1327,...,50.95,924,-50,6677,4703.0,32,363,19.87,0.03311,1.09
8,2020-11-22T17:00:00,19,Sicilia,1597,241,1838,35324,37162,921,1258,...,40.99,893,-580,6447,3887.0,45,292,19.51,0.02532,1.07
9,2020-11-22T17:00:00,4,Trentino-Alto Adige,873,79,952,13214,14166,-211,694,...,74.53,-221,-66,6424,1844.0,8,897,10.80,0.06457,3.29


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,949,87,1036,91.60
1,Trentino-Alto Adige,79,27,106,74.53
2,Valle d'Aosta,14,6,20,70.00
3,Piemonte,398,177,575,69.22
4,Umbria,75,42,117,64.10
5,Marche,88,55,143,61.54
6,Liguria,122,87,209,58.37
7,Toscana,298,225,523,56.98
8,Puglia,188,181,369,50.95
9,Abruzzo,72,76,148,48.65


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,2020-11-20T17:00:00,22,P.A. Trento,417,38,455,2549,3004,18,234,10194,563,7178.0,6583.0,13761,344029,125701.0
542,2020-11-21T17:00:00,21,P.A. Bolzano,450,39,489,11186,11675,164,548,8528,462,20665.0,0.0,20665,291617,143675.0
543,2020-11-21T17:00:00,22,P.A. Trento,413,40,453,2249,2702,-302,212,10694,577,7252.0,6721.0,13973,348903,127347.0
544,2020-11-22T17:00:00,21,P.A. Bolzano,453,38,491,11133,11624,-51,535,9107,469,21200.0,0.0,21200,294506,144616.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,2020-11-18T17:00:00,4,Trentino-Alto Adige,879,77,956,12805,13761,66,817,...,18.0,7692.0,1965.0,59.0,8.05,6.95,10.62,41.58,-2.53,5.966
269,2020-11-19T17:00:00,4,Trentino-Alto Adige,876,78,954,13236,14190,429,962,...,20.0,6135.0,1790.0,431.0,8.18,6.72,15.68,53.74,1.30,6.991
270,2020-11-20T17:00:00,4,Trentino-Alto Adige,885,76,961,13554,14515,325,970,...,15.0,6330.0,1663.0,318.0,7.91,6.62,15.32,58.33,-2.56,6.836
271,2020-11-21T17:00:00,4,Trentino-Alto Adige,863,79,942,13435,14377,-138,760,...,30.0,7784.0,2663.0,-119.0,8.39,6.55,9.76,28.54,3.95,5.236


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
